In [1]:
!pip3 install python-dotenv
!pip3 install transformers --q
!pip3 install torch --q


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python3.11 -m pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python3.11 -m pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

HF_TOKEN = os.getenv('HF_TOKEN')

In [3]:
from transformers import AutoModel
from numpy.linalg import norm

cos_sim = lambda a,b: (a @ b.T) / (norm(a)*norm(b))
model = AutoModel.from_pretrained('jinaai/jina-embeddings-v2-base-en', trust_remote_code=True) # trust_remote_code is needed to use the encode method

A new version of the following files was downloaded from https://huggingface.co/jinaai/jina-bert-implementation:
- configuration_bert.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_bert.py:   0%|          | 0.00/97.7k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/jinaai/jina-bert-implementation:
- modeling_bert.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


In [4]:
import re
def clean(text):
    pattern = re.compile(r'[^a-zA-Z\s]')
    return re.sub(pattern, '', text)

In [5]:
from tweetretriever import TweetRetriever
tr = TweetRetriever()

In [6]:
dylan_posts = tr.loadFromFile('../dylan_posts.json')
id_map = {tr.getTweetID(tweet):tweet for tweet in dylan_posts}
for t in dylan_posts:
    if 'tweetText' in t:
        t['tweetText'] = clean(t['tweetText'])
tweet_texts = {tr.getTweetID(tweet):tweet['tweetText'] for tweet in dylan_posts if 'tweetText' in tweet}

FileNotFoundError: [Errno 2] No such file or directory: '../dylan_posts.json'

In [12]:
from itertools import combinations
from tqdm import tqdm
pairs = list(combinations(tweet_texts,2))
res = [] 
for k1,k2 in tqdm(pairs):
    text1 = tweet_texts[k1]
    text2 = tweet_texts[k2]
    embeddings = model.encode([text1,text2],max_length=max(len(text1),len(text2)))
    res.append((k1,k2,cos_sim(*embeddings)))

100%|██████████| 1596/1596 [01:53<00:00, 14.06it/s]


In [13]:
len(res)

1596

In [ ]:
ordered = sorted(list(set(res)),key=lambda x:-x[-1])
ordered = [(id_map[k1],id_map[k2],score) for (k1,k2,score) in ordered]
with open('../analyzed_data/shareable/dylan_similar_tweets.txt','x') as f:
    f.write("Dylan's 15 Most Similar Tweets")
    f.write('\n\n\n')
    for i,tweets in enumerate(ordered[:15]):
        t1 = tr.prettifyForWrite(tweets[0])
        t2 = tr.prettifyForWrite(tweets[1])
        score = tweets[2]
        f.write(f'{i+1}.\n')
        f.write(t1)
        f.write('\n')
        f.write(t2)
        f.write('\n')
        f.write(f'Score: {score}\n\n')
    f.write('\n')
